In [1]:
import tensorflow as tf
from tensorflow import keras

In [2]:
print('numberof gpus: ',len(tf.config.experimental.list_physical_devices('GPU')))

numberof gpus:  1


In [5]:

################### RUN THIS CELL

############ Manaaaaan
# Load the Drive helper and mount
from google.colab import drive
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model

# This will prompt for authorization.
drive.mount('/MyDrive')

import numpy as np
import pandas as pd
import tensorflow as tf
import os,cv2

import warnings
warnings.filterwarnings("ignore")

data_path = '../MyDrive/MyDrive/Dataset'


labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107



train_videos=[]
train_labels=[]
import traceback
import random

num = 1

for i,folder in enumerate(labels):
    try:
      for video in os.listdir(data_path+'/'+folder):
        print(num)
        num+=1
        video = os.path.join(data_path+'/'+folder+'/'+video)
        # Open the video file
        cap = cv2.VideoCapture(video)

        # Get the total number of frames in the video
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Select a random starting frame number
        start_frame = random.randint(0, total_frames - 16)

        # Set the number of frames to be selected
        num_frames = 16

        # Set the frame number to start with
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        # Loop through the frames and save them
        frames = []
        for j in range(num_frames):
            ret, image = cap.read()
            if ret:
              frames.append(cv2.resize(image,(112,112)))

        # Release the video file
        cap.release()

        train_videos.append(frames)
        train_labels.append(i)

    except Exception:
      traceback.print_exc()
train_videos = np.asarray(train_videos)
train_labels = np.asarray(train_labels).astype('int64')

# import numpy as np
# arr_reshaped = train_videos.reshape(train_videos.shape[0], -1) #1151
# np.savetxt("dataset.txt", arr_reshaped)

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_videos,train_labels,test_size=0.25,shuffle=True)

input_shape =(32,16,112,112,3)
output_shape = 20

tf.keras.regularizers.L1L2(
    l1=0.01, l2=0.1
)

RGBmodel = Sequential()

# model.add(RandomFlip("horizontal"))
# model.add(RandomRotation(0.1))
# RGBmodel.add(Rescaling(1.0 / 255,input_shape=input_shape[1:]))
RGBmodel.add(Conv3D(64,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same',input_shape=input_shape[1:]))
RGBmodel.add(MaxPooling3D(pool_size=(1, 2, 2),strides =(1,2,2)))    #maxPool-1

RGBmodel.add(Conv3D(128,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-2
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(256,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-3a
RGBmodel.add(Conv3D(256,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-3b
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-4a
RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-4b
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-5a
RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-5b

RGBmodel.add(ZeroPadding3D(padding=(0,1,1)))
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))
RGBmodel.add(Flatten())

RGBmodel.add(Dropout(rate=0.5))
RGBmodel.add(Dense(units=4096, activation='relu', kernel_regularizer='l2'))
RGBmodel.add(Dropout(rate=0.5))
RGBmodel.add(Dense(units=4096, activation='relu', kernel_regularizer='l1'))


RGBmodel.add(Dense(units = output_shape, activation = 'softmax'))

from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint

# define your model here


# set up the checkpointing callback
checkpoint = ModelCheckpoint("model_weights.h5", save_weights_only=True)

# train the model for 100 epochs and save the weights
opt = SGD(learning_rate=0.001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2, callbacks=[checkpoint])

# load the saved weights and continue training for another 100 epochs
opt = SGD(learning_rate=0.001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.load_weights("model_weights.h5")
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2, callbacks=[checkpoint])

# repeat the previous step until you have trained for 500 epochs
opt = SGD(learning_rate=0.001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.load_weights("model_weights.h5")
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2, callbacks=[checkpoint])

opt = SGD(learning_rate=0.001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.load_weights("model_weights.h5")
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2, callbacks=[checkpoint])

opt = SGD(learning_rate=0.0001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.load_weights("model_weights.h5")
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2, callbacks=[checkpoint])


# Print the summary of the model
RGBmodel.summary()
RGBmodel.save('RGBmodel.h5') 

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
RGBmodel = Sequential()

# model.add(RandomFlip("horizontal"))
# model.add(RandomRotation(0.1))
# RGBmodel.add(Rescaling(1.0 / 255,input_shape=input_shape[1:]))
RGBmodel.add(Conv3D(64,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same',input_shape=input_shape[1:]))
RGBmodel.add(MaxPooling3D(pool_size=(1, 2, 2),strides =(1,2,2)))    #maxPool-1

RGBmodel.add(Conv3D(128,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-2
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(256,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-3a
RGBmodel.add(Conv3D(256,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-3b
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-4a
RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-4b
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-5a
RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-5b

RGBmodel.add(ZeroPadding3D(padding=(0,1,1)))
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))
RGBmodel.add(Flatten())

RGBmodel.add(Dropout(rate=0.5))
RGBmodel.add(Dense(units=4096, activation='relu', kernel_regularizer='l2'))
RGBmodel.add(Dropout(rate=0.5))
RGBmodel.add(Dense(units=4096, activation='relu', kernel_regularizer='l1'))


RGBmodel.add(Dense(units = output_shape, activation = 'softmax'))

from keras.optimizers import SGD
opt = SGD(learning_rate=0.001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 400, validation_split = 0.2)


from keras.optimizers import SGD
opt = SGD(learning_rate=0.0001)s
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2)
RGBmodel.save('RGBmodel.h5') 

In [ ]:
from keras.callbacks import ModelCheckpoint

# define your model here


# set up the checkpointing callback
checkpoint = ModelCheckpoint("model_weights.h5", save_weights_only=True)

# train the model for 100 epochs and save the weights
opt = SGD(learning_rate=0.001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2, callbacks=[checkpoint])

# load the saved weights and continue training for another 100 epochs
opt = SGD(learning_rate=0.001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.load_weights("model_weights.h5")
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2, callbacks=[checkpoint])

# repeat the previous step until you have trained for 500 epochs
opt = SGD(learning_rate=0.001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.load_weights("model_weights.h5")
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2, callbacks=[checkpoint])

opt = SGD(learning_rate=0.001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.load_weights("model_weights.h5")
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2, callbacks=[checkpoint])

opt = SGD(learning_rate=0.0001)s
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.load_weights("model_weights.h5")
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2, callbacks=[checkpoint])


In [ ]:
RGBmodel.save('RGBmodel.h5') 

In [2]:

################### RUN THIS CELL


# Load the Drive helper and mount
# from google.colab import drive
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras import initializers
from keras.layers import Conv3D, MaxPooling3D,Dense,ZeroPadding3D,Flatten,Activation,BatchNormalization,RandomFlip,RandomRotation,Rescaling
from keras.models import load_model

# This will prompt for authorization.
# drive.mount('/MyDrive')

import numpy as np
import pandas as pd
import tensorflow as tf
import os,cv2

import warnings
warnings.filterwarnings("ignore")

# data_path = '../MyDrive/MyDrive/Dataset'

data_path = './Dataset'
labels=[]
for folder in os.listdir(data_path):
    labels.append(folder)
labels.sort() #len = 107



train_videos=[]
train_labels=[]
import traceback
import random

num = 1

for i,folder in enumerate(labels):
    try:
      for video in os.listdir(data_path+'/'+folder):
        print(num)
        num+=1
        video = os.path.join(data_path+'/'+folder+'/'+video)
        # Open the video file
        cap = cv2.VideoCapture(video)

        # Get the total number of frames in the video
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Select a random starting frame number
        start_frame = random.randint(0, total_frames - 16)

        # Set the number of frames to be selected
        num_frames = 16

        # Set the frame number to start with
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

        # Loop through the frames and save them
        frames = []
        for j in range(num_frames):
            ret, image = cap.read()
            if ret:
              frames.append(cv2.resize(image,(112,112)))

        # Release the video file
        cap.release()

        train_videos.append(frames)
        train_labels.append(i)

    except Exception:
      traceback.print_exc()
train_videos = np.asarray(train_videos)
train_labels = np.asarray(train_labels).astype('int64')


# from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test = train_test_split(train_videos,train_labels,test_size=0.25,shuffle=True)

# input_shape =(32,16,112,112,3)
# output_shape = 20

# tf.keras.regularizers.L1L2(
#     l1=0.01, l2=0.1
# )

# RGBmodel = Sequential()

# # model.add(RandomFlip("horizontal"))
# # model.add(RandomRotation(0.1))
# # RGBmodel.add(Rescaling(1.0 / 255,input_shape=input_shape[1:]))
# RGBmodel.add(Conv3D(64,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same',input_shape=input_shape[1:]))
# RGBmodel.add(MaxPooling3D(pool_size=(1, 2, 2),strides =(1,2,2)))    #maxPool-1

# RGBmodel.add(Conv3D(128,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-2
# RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

# RGBmodel.add(Conv3D(256,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-3a
# RGBmodel.add(Conv3D(256,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-3b
# RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

# RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-4a
# RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-4b
# RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

# RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-5a
# RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-5b

# RGBmodel.add(ZeroPadding3D(padding=(0,1,1)))
# RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))
# RGBmodel.add(Flatten())

# RGBmodel.add(Dropout(rate=0.5))
# RGBmodel.add(Dense(units=4096, activation='relu', kernel_regularizer='l2'))
# RGBmodel.add(Dropout(rate=0.5))
# RGBmodel.add(Dense(units=4096, activation='relu', kernel_regularizer='l1'))


# RGBmodel.add(Dense(units = output_shape, activation = 'softmax'))

# from keras.optimizers import SGD
# opt = SGD(learning_rate=0.001)
# RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
# RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 400, validation_split = 0.2)


# from keras.optimizers import SGD
# opt = SGD(learning_rate=0.0001)
# RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
# RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2)

# # Print the summary of the model
# RGBmodel.summary()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:

from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(train_videos,train_labels,test_size=0.25,shuffle=True)

input_shape =(32,16,112,112,3)
output_shape = 20

tf.keras.regularizers.L1L2(
    l1=0.01, l2=0.1
)

RGBmodel = Sequential()

# model.add(RandomFlip("horizontal"))
# model.add(RandomRotation(0.1))
# RGBmodel.add(Rescaling(1.0 / 255,input_shape=input_shape[1:]))
RGBmodel.add(Conv3D(64,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same',input_shape=input_shape[1:]))
RGBmodel.add(MaxPooling3D(pool_size=(1, 2, 2),strides =(1,2,2)))    #maxPool-1

RGBmodel.add(Conv3D(128,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-2
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(256,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-3a
RGBmodel.add(Conv3D(256,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-3b
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-4a
RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-4b
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))

RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-5a
RGBmodel.add(Conv3D(512,kernel_size=(3,3,3),strides=(1, 1, 1),activation='relu',padding = 'same')) #conv3d-5b

RGBmodel.add(ZeroPadding3D(padding=(0,1,1)))
RGBmodel.add(MaxPooling3D(pool_size=(2, 2, 2),strides =(2,2,2)))
RGBmodel.add(Flatten())

RGBmodel.add(Dropout(rate=0.5))
RGBmodel.add(Dense(units=4096, activation='relu', kernel_regularizer='l2'))
RGBmodel.add(Dropout(rate=0.5))
RGBmodel.add(Dense(units=4096, activation='relu', kernel_regularizer='l1'))


RGBmodel.add(Dense(units = output_shape, activation = 'softmax'))

from tensorflow.keras.optimizers import SGD
opt = SGD(learning_rate=0.001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 400, validation_split = 0.2)


from tensorflow.keras.optimizers import SGD
opt = SGD(learning_rate=0.0001)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 100, validation_split = 0.2)

# Print the summary of the model
RGBmodel.summary()

Epoch 1/400
23/23 [==============================] - 66s 1s/step - loss: 2309.3901 - accuracy: 0.0371 - val_loss: 2288.7871 - val_accuracy: 0.0275
Epoch 2/400
23/23 [==============================] - 13s 589ms/step - loss: 2270.7314 - accuracy: 0.0371 - val_loss: 2250.6379 - val_accuracy: 0.0275
Epoch 3/400
23/23 [==============================] - 13s 588ms/step - loss: 2232.7400 - accuracy: 0.0523 - val_loss: 2212.8042 - val_accuracy: 0.0275
Epoch 4/400
23/23 [==============================] - 14s 589ms/step - loss: 2195.0586 - accuracy: 0.0523 - val_loss: 2175.3330 - val_accuracy: 0.0275
Epoch 5/400
23/23 [==============================] - 14s 589ms/step - loss: 2157.7351 - accuracy: 0.0591 - val_loss: 2138.1660 - val_accuracy: 0.0275
Epoch 6/400
23/23 [==============================] - 14s 590ms/step - loss: 2120.7339 - accuracy: 0.0385 - val_loss: 2101.3384 - val_accuracy: 0.0275
Epoch 7/400
23/23 [==============================] - 14s 591ms/step - loss: 2084.0608 - accuracy: 0.061

23/23 [==============================] - 14s 598ms/step - loss: 65.1390 - accuracy: 0.2558 - val_loss: 63.6678 - val_accuracy: 0.2857
Epoch 111/400
23/23 [==============================] - 14s 598ms/step - loss: 62.6546 - accuracy: 0.2696 - val_loss: 61.3995 - val_accuracy: 0.3077
Epoch 112/400
23/23 [==============================] - 14s 598ms/step - loss: 60.5157 - accuracy: 0.2655 - val_loss: 59.7290 - val_accuracy: 0.2033
Epoch 113/400
23/23 [==============================] - 14s 598ms/step - loss: 58.7249 - accuracy: 0.2586 - val_loss: 58.3048 - val_accuracy: 0.1813
Epoch 114/400
23/23 [==============================] - 14s 598ms/step - loss: 57.2314 - accuracy: 0.2613 - val_loss: 56.4656 - val_accuracy: 0.2582
Epoch 115/400
23/23 [==============================] - 14s 598ms/step - loss: 56.0669 - accuracy: 0.2462 - val_loss: 55.8428 - val_accuracy: 0.1923
Epoch 116/400
23/23 [==============================] - 14s 598ms/step - loss: 55.2263 - accuracy: 0.2201 - val_loss: 55.0142 -

23/23 [==============================] - 14s 600ms/step - loss: 48.1505 - accuracy: 0.4250 - val_loss: 48.5695 - val_accuracy: 0.3187
Epoch 221/400
23/23 [==============================] - 14s 599ms/step - loss: 48.1511 - accuracy: 0.4539 - val_loss: 48.5085 - val_accuracy: 0.3352
Epoch 222/400
23/23 [==============================] - 14s 599ms/step - loss: 48.1712 - accuracy: 0.4470 - val_loss: 48.2348 - val_accuracy: 0.3681
Epoch 223/400
23/23 [==============================] - 14s 599ms/step - loss: 48.1055 - accuracy: 0.4470 - val_loss: 48.2943 - val_accuracy: 0.3352
Epoch 224/400
23/23 [==============================] - 14s 599ms/step - loss: 48.0226 - accuracy: 0.4182 - val_loss: 48.2940 - val_accuracy: 0.3626
Epoch 225/400
23/23 [==============================] - 14s 599ms/step - loss: 47.9797 - accuracy: 0.4374 - val_loss: 48.3456 - val_accuracy: 0.2967
Epoch 226/400
23/23 [==============================] - 14s 599ms/step - loss: 47.9615 - accuracy: 0.4360 - val_loss: 48.0973 -

23/23 [==============================] - 14s 599ms/step - loss: 43.4180 - accuracy: 0.6162 - val_loss: 43.9604 - val_accuracy: 0.4341
Epoch 331/400
23/23 [==============================] - 14s 600ms/step - loss: 43.4650 - accuracy: 0.6011 - val_loss: 43.8554 - val_accuracy: 0.4396
Epoch 332/400
23/23 [==============================] - 14s 599ms/step - loss: 43.2513 - accuracy: 0.6245 - val_loss: 43.7297 - val_accuracy: 0.4890
Epoch 333/400
23/23 [==============================] - 14s 598ms/step - loss: 43.2660 - accuracy: 0.6217 - val_loss: 43.7063 - val_accuracy: 0.4505
Epoch 334/400
23/23 [==============================] - 14s 598ms/step - loss: 43.2352 - accuracy: 0.6204 - val_loss: 43.7198 - val_accuracy: 0.4176
Epoch 335/400
23/23 [==============================] - 14s 600ms/step - loss: 43.1541 - accuracy: 0.6162 - val_loss: 43.5540 - val_accuracy: 0.4341
Epoch 336/400
23/23 [==============================] - 14s 598ms/step - loss: 43.1765 - accuracy: 0.6094 - val_loss: 43.7958 -

Epoch 41/100
23/23 [==============================] - 14s 598ms/step - loss: 38.4480 - accuracy: 0.8253 - val_loss: 39.4643 - val_accuracy: 0.5165
Epoch 42/100
23/23 [==============================] - 14s 599ms/step - loss: 38.4755 - accuracy: 0.8336 - val_loss: 39.4503 - val_accuracy: 0.5000
Epoch 43/100
23/23 [==============================] - 14s 599ms/step - loss: 38.4571 - accuracy: 0.8226 - val_loss: 39.4588 - val_accuracy: 0.5055
Epoch 44/100
23/23 [==============================] - 14s 599ms/step - loss: 38.4088 - accuracy: 0.8597 - val_loss: 39.4555 - val_accuracy: 0.5000
Epoch 45/100
23/23 [==============================] - 14s 599ms/step - loss: 38.4428 - accuracy: 0.8322 - val_loss: 39.4804 - val_accuracy: 0.5055
Epoch 46/100
23/23 [==============================] - 14s 612ms/step - loss: 38.4290 - accuracy: 0.8418 - val_loss: 39.4217 - val_accuracy: 0.5220
Epoch 47/100
23/23 [==============================] - 14s 604ms/step - loss: 38.4111 - accuracy: 0.8501 - val_loss: 39

Epoch 97/100
23/23 [==============================] - 14s 602ms/step - loss: 38.2156 - accuracy: 0.8446 - val_loss: 39.3181 - val_accuracy: 0.5110
Epoch 98/100
23/23 [==============================] - 14s 598ms/step - loss: 38.2038 - accuracy: 0.8652 - val_loss: 39.3403 - val_accuracy: 0.5055
Epoch 99/100
23/23 [==============================] - 14s 601ms/step - loss: 38.2183 - accuracy: 0.8404 - val_loss: 39.3248 - val_accuracy: 0.5055
Epoch 100/100
23/23 [==============================] - ETA: 0s - loss: 38.1977 - accuracy: 0.8583

In [ ]:
RGBmodel.save('RGBStream.h5')

In [ ]:
RGBmodel.ev

In [ ]:
from tensorflow.keras.optimizers import SGD
opt = SGD(learning_rate=0.0005)
RGBmodel.compile(loss = 'sparse_categorical_crossentropy',optimizer = opt, metrics = ['accuracy'])
RGBmodel.fit(X_train,y_train,batch_size=32,epochs = 50, validation_split = 0.2)